In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas
import xarray
import cftime
import tensorflow as tf

/Users/paulag/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
datapath = 'nino34_monthly.nc'
nino34 = xarray.open_dataset(datapath, decode_times = False)
print(nino34)
nino34 = np.array(nino34['nino34'].values)

<xarray.Dataset>
Dimensions:         (bounds: 2, month: 12, time: 7800)
Coordinates:
  * time            (time) float64 15.5 45.0 74.5 ... 2.372e+05 2.372e+05
Dimensions without coordinates: bounds, month
Data variables:
    nino34          (time) float64 ...
    time_bnds       (time, bounds) float64 ...
    areacello       float32 ...
    days_per_month  (month) int32 ...


In [3]:
def ONI(nino34, m = 3):
    oni = np.array(nino34)
    length = nino34.shape[0]
    for i in range(length):
        oni[i] = np.mean(nino34[max(0, (i - m + 1)) : min((i + 1), length)])
    return oni

In [4]:
oni = ONI(nino34)

In [5]:
def climatology(nino34):
    clm = np.zeros(12)
    length = nino34.shape[0]
    for month in range(12):
        section = [12 * i + month for i in range(length // 12)]
        clm[month] = np.mean(nino34[section])
    return clm

In [6]:
clm = climatology(nino34)

In [7]:
def SST_anomaly(nino34, clm):
    anm = np.array(nino34)
    length = nino34.shape[0]
    for i in range(length):
        anm[i] = nino34[i] - clm[i % 12]
    return anm

In [8]:
anm = SST_anomaly(nino34, clm)
oanm = ONI(anm)

# Data Preparation

In [9]:
T = 6                       # prediction timeline
H = 48                      # history used for prediction
include_month = 0           # 1 if we use the month as a feature, 0 otherwise
n_classes = 3               # number of classes (El Nino, El Nina, No Event)
threshold = 0.5         
signal = np.array(nino34)   # data used for training/testing
length = signal.shape[0]    # number of data points
size = length - H - T       # effective dataset size

In [10]:
# create the 'history matrix'
data = np.ndarray((size, H + include_month))
for i in range(size):
    if(include_month == False):
        data[i] = signal[i:(i + H)]
    else:
        data[i] = np.append(signal[i:(i + H)], (i + H + T) % 12)

# label El Nino as 2, El Nina as 0 and no event as 1
labels = np.ndarray((size))
for i in range(length - H - T):
    if(oanm[i + H + T] >= threshold):
        labels[i] = 2
    elif(oanm[i + H + T] <= -threshold):
        labels[i] = 0
    else:
        labels[i] = 1

We split the data into 80% training, 10% validation and 10% testing.

In [11]:
# predicting february

np.random.seed(0)

size2 = size // 12
    
split = size2 // 10      
shuffle = 12 * np.random.permutation(size2) + np.ones(size2, dtype = int)
    
train_ind = np.array(shuffle[0: 8 * split])
val_ind = np.array(shuffle[8 * split: 9 * split])
test_ind = np.array(shuffle[9 * split: size2])

train = np.array(data[train_ind])
train_labels = np.array(labels[train_ind])

val = np.array(data[val_ind])
val_labels = np.array(labels[val_ind])

test = np.array(data[test_ind])
test_labels = np.array(labels[test_ind])
    
mean = np.mean(train, axis = 0)
std = np.std(train, axis = 0)

train_n = np.divide(
        train - np.outer(np.ones(train.shape[0]), mean),
        np.outer(np.ones(train.shape[0]), std))

val_n = np.divide(
        val - np.outer(np.ones(val.shape[0]), mean),
        np.outer(np.ones(val.shape[0]), std))

test_n = np.divide(
        test - np.outer(np.ones(test.shape[0]), mean),
        np.outer(np.ones(test.shape[0]), std))

with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        encoded_train_labels = tf.one_hot(train_labels, depth = n_classes).eval()
        encoded_val_labels = tf.one_hot(val_labels, depth = n_classes).eval()
        encoded_test_labels = tf.one_hot(test_labels, depth = n_classes).eval()

# Optimizing Deep Neural Network Architecture

We will fix the number of epochs to 25 and vary dropout, the number of layers and the number of nodes.

In [12]:
epochs = 25
loss = {}
acc = {}

In [15]:
for l in [1, 2, 3, 4]:
    for i in [1, 2, 4, 8]:
        for d in [0.2, 0.3, 0.5]:
            
            N = i * H
            
            layers = []
            
            for n in range(l): 
                layers.append(tf.keras.layers.Dense(N, activation=tf.nn.relu))
                layers.append(tf.keras.layers.Dropout(d))
            
            layers.append(tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax))
            
            model = tf.keras.models.Sequential(layers)

            model.reset_states()

            model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                    metrics=['accuracy'])

            model.fit(train_n, encoded_train_labels, epochs = epochs, verbose = 0)
            (loss[(l, i, d)], acc[(l, i, d)]) = model.evaluate(val_n, encoded_val_labels, verbose = 0)
            print(l, " layers, ", N, " nodes, ", d, " dropout", acc[(l, i, d)], "% accuracy")

1  layers,  48  nodes,  0.2  dropout 0.71875 % accuracy
1  layers,  48  nodes,  0.3  dropout 0.609375 % accuracy
1  layers,  48  nodes,  0.5  dropout 0.640625 % accuracy
1  layers,  96  nodes,  0.2  dropout 0.640625 % accuracy
1  layers,  96  nodes,  0.3  dropout 0.71875 % accuracy
1  layers,  96  nodes,  0.5  dropout 0.703125 % accuracy
1  layers,  192  nodes,  0.2  dropout 0.6875 % accuracy
1  layers,  192  nodes,  0.3  dropout 0.671875 % accuracy
1  layers,  192  nodes,  0.5  dropout 0.671875 % accuracy
1  layers,  384  nodes,  0.2  dropout 0.671875 % accuracy
1  layers,  384  nodes,  0.3  dropout 0.65625 % accuracy
1  layers,  384  nodes,  0.5  dropout 0.625 % accuracy
2  layers,  48  nodes,  0.2  dropout 0.671875 % accuracy
2  layers,  48  nodes,  0.3  dropout 0.640625 % accuracy
2  layers,  48  nodes,  0.5  dropout 0.671875 % accuracy
2  layers,  96  nodes,  0.2  dropout 0.640625 % accuracy
2  layers,  96  nodes,  0.3  dropout 0.671875 % accuracy
2  layers,  96  nodes,  0.5  drop

# Number of nodes in layer

In [12]:
loss = {}
acc = {}

In [13]:
for e in [10, 15, 20, 25, 30]:
    for N in np.arange(8, 128, 16, dtype = int):
        for d in [0.2, 0.3, 0.5]:
            
            model = tf.keras.models.Sequential([
              tf.keras.layers.Dense(N, activation=tf.nn.relu),
              tf.keras.layers.Dropout(d),
              tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
            ])

            model.reset_states()

            model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                    metrics=['accuracy'])

            model.fit(train_n, encoded_train_labels, epochs = e, verbose = 0)
            (loss[(e, N, d)], acc[(e, N, d)]) = model.evaluate(val_n, encoded_val_labels, verbose = 0)
            print(e, " epochs, ", N, " nodes, ", d, " dropout", acc[(e, N, d)], "% accuracy")

10  epochs,  8  nodes,  0.2  dropout 0.5625 % accuracy
10  epochs,  8  nodes,  0.3  dropout 0.640625 % accuracy
10  epochs,  8  nodes,  0.5  dropout 0.609375 % accuracy
10  epochs,  24  nodes,  0.2  dropout 0.65625 % accuracy
10  epochs,  24  nodes,  0.3  dropout 0.625 % accuracy
10  epochs,  24  nodes,  0.5  dropout 0.578125 % accuracy
10  epochs,  40  nodes,  0.2  dropout 0.578125 % accuracy
10  epochs,  40  nodes,  0.3  dropout 0.609375 % accuracy
10  epochs,  40  nodes,  0.5  dropout 0.671875 % accuracy
10  epochs,  56  nodes,  0.2  dropout 0.640625 % accuracy
10  epochs,  56  nodes,  0.3  dropout 0.671875 % accuracy
10  epochs,  56  nodes,  0.5  dropout 0.59375 % accuracy
10  epochs,  72  nodes,  0.2  dropout 0.640625 % accuracy
10  epochs,  72  nodes,  0.3  dropout 0.65625 % accuracy
10  epochs,  72  nodes,  0.5  dropout 0.65625 % accuracy
10  epochs,  88  nodes,  0.2  dropout 0.640625 % accuracy
10  epochs,  88  nodes,  0.3  dropout 0.625 % accuracy
10  epochs,  88  nodes,  0.5 